In [300]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import datetime
import pickle
import re

import lxml.etree as etree
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
stop_words_list = stopwords.words('english')
ns = {"mods": "http://www.loc.gov/mods/v3"}

from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [2]:
model = BertForSequenceClassification.from_pretrained("data/b-druids-model")

In [41]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [5]:
etds_path = pathlib.Path("../tmp/etd-10-27/results/")

In [7]:
bio_druids = pd.read_csv("data/druids_biology_cleaned.csv")

In [9]:
bio_druids.sample()

,druids
603,sh172dm7469


In [10]:
druid_603 = etds_path/'sh172dm7469.txt'

In [16]:
bio_fast = pd.read_csv("../biology-etds/data/fast-bio.csv")

In [17]:
bio_fast.tail()

,id,URI,Label
1788,1788,http://id.worldcat.org/fast/235269,"Amici, Giovanni Battista, 1786-1863"
1789,1789,http://id.worldcat.org/fast/185500,"Clark, William Andrews, 1877-1934"
1790,1790,http://id.worldcat.org/fast/675822,Nationaal Natuurhistorisch Museum (Netherlands)
1791,1791,http://id.worldcat.org/fast/444817,"Menzies, Archibald, 1754-1842"
1792,1792,http://id.worldcat.org/fast/43064,"Linné, Carl von, 1707-1778"


In [18]:
def batch(full_text: str) -> list:
    batches = []
    for i in range(0, len(full_text), 512):
        batches.append(full_text[i:i+512])
    return batches

In [19]:
druid_603_batches = batch(druid_603.read_text())

In [20]:
len(druid_603_batches)

342

In [33]:
druid_603_data = []
zeros = list(np.zeros(1793))
for row in druid_603_batches:
    record = ['sh172dm7469', row]
    record.extend(zeros)
    druid_603_data.append(record)
    

In [59]:
column_names = ['druid', 'text']
column_names.extend(list(bio_fast['URI']))
druid_603_df = pd.DataFrame(druid_603_data, columns=column_names)

In [60]:
druid_603_label_cols = list(druid_603_df.columns[2:])

In [61]:
druid_603_df['one_hot_labels'] = list(druid_603_df[druid_603_label_cols].values)

In [62]:
druid_603_df.head()

,druid,text,http://id.worldcat.org/fast/870268,http://id.worldcat.org/fast/894932,http://id.worldcat.org/fast/917265,http://id.worldcat.org/fast/887377,http://id.worldcat.org/fast/897386,http://id.worldcat.org/fast/1204623,http://id.worldcat.org/fast/1205427,http://id.worldcat.org/fast/1065823,...,http://id.worldcat.org/fast/179858,http://id.worldcat.org/fast/434309,http://id.worldcat.org/fast/1423826,http://id.worldcat.org/fast/1423871,http://id.worldcat.org/fast/235269,http://id.worldcat.org/fast/185500,http://id.worldcat.org/fast/675822,http://id.worldcat.org/fast/444817,http://id.worldcat.org/fast/43064,one_hot_labels
0,sh172dm7469,IDENTIFYING LOCATIONS AND CHARACTERIZING CIS\n...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,sh172dm7469,University under license with the author.\n ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,sh172dm7469,issertation for the degree of Doctor of Philos...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,sh172dm7469,ality as a dissertation for the degree of Doct...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,sh172dm7469,"Patricia J. Gumport, Vice Provost for Graduat...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [63]:
druid_603_hot_labels = list(druid_603_df.one_hot_labels.values)
druid_603_text = list(druid_603_df.text.values)

In [65]:
druid_603_encodings = tokenizer.batch_encode_plus(druid_603_text, max_length=512, pad_to_max_length=True, truncation=True)

/Users/jpnelson/2020/py3-env/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1767: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [66]:
druid_603_input_ids = druid_603_encodings['input_ids']
druid_603_token_type_ids = druid_603_encodings['token_type_ids']
druid_603_attention_masks = druid_603_encodings['attention_mask']

In [70]:
druid_603_inputs = torch.tensor(druid_603_input_ids)
druid_603_labels = torch.tensor(druid_603_hot_labels)
druid_603_masks = torch.tensor(druid_603_attention_masks)
druid_603_token_types = torch.tensor(druid_603_token_type_ids)

druid_603_data = TensorDataset(druid_603_inputs, druid_603_masks, druid_603_labels, druid_603_token_types)
druid_603_sampler = SequentialSampler(druid_603_data)
druid_603_dataloader = DataLoader(druid_603_data, sampler=druid_603_sampler, batch_size=8)

In [75]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
start = datetime.datetime.utcnow()
print(f"Started evaluation at {start}")
model.eval()
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]
for i, batch in enumerate(druid_603_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    while torch.no_grad():
        outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        b_logit_pred = outs[0]
        pred_label = torch.sigmoid(b_logit_pred)

        b_logit_pred = b_logit_pred.detach().cpu().numpy()
        pred_label = pred_label.to('cpu').detach().numpy()
#         b_labels = b_labels.to('cpu').detach().numpy()
    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)
    batch_date = datetime.datetime.utcnow()
    print(f"{i} finished at {batch_date}, time elapsed {(batch_data-start).seconds / 60.}")
# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]
pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]
# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]
end = datetime.datetime.utcnow()
print(f"Finished Evaluation at {end}, elapsed time {(end-start).seconds / 60.}")

Started evaluation at 2020-10-28 22:54:55.765087


KeyboardInterrupt: 

In [84]:
import re
special_char_re = re.compile(r'[^a-zA-Z]')
from nltk.corpus import stopwords

stop_words_list = stopwords.words('english')
def cleanup(term: str) -> str:
    cleaned = []
    for char in term.split():
        cleaned_char = special_char_re.sub(' ', char).lower()
        if cleaned_char in stop_words_list:
            continue
        cleaned.append(cleaned_char)
    return ' '.join(cleaned)

In [77]:
chron_df = pd.read_csv("data/chron_uri_label_utf8.csv", names=["URI", "Label"])

In [85]:
chron_df['clean'] = chron_df['Label'].apply(cleanup)

In [86]:
chron_df.head()

,URI,Label,clean
0,http://id.worldcat.org/fast/1356000,963-969,
1,http://id.worldcat.org/fast/1355997,960-1386,
2,http://id.worldcat.org/fast/1355744,1773-1796,
3,http://id.worldcat.org/fast/1355919,37-41,
4,http://id.worldcat.org/fast/1355499,1371-1707,


In [87]:
topics_df = pd.read_csv("data/topic_uri_label_utf8.csv", names=["URI", "Label"])
topics_df["cleaned"] = topics_df["Label"].apply(cleanup)

In [89]:
topics_df.head()

,URI,Label,cleaned
0,http://id.worldcat.org/fast/799409,African American teenagers--Education,african american teenagers education
1,http://id.worldcat.org/fast/912703,Enthusiasm--Religious aspects--Christianity,enthusiasm religious aspects christianity
2,http://id.worldcat.org/fast/966912,Identity (Psychology) in old age,identity psychology old age
3,http://id.worldcat.org/fast/817698,Artists' studios,artists studios
4,http://id.worldcat.org/fast/833340,Bisexual women--Health and hygiene,bisexual women health hygiene


In [90]:
geo_df = pd.read_csv("data/geo_uri_label_utf8.csv", names=["URI", "Label"])
geo_df["cleaned"] = geo_df["Label"].apply(cleanup)

In [91]:
geo_df.head()

,URI,Label,cleaned
0,http://id.worldcat.org/fast/1891285,North America--Dalton Trail,north america dalton trail
1,http://id.worldcat.org/fast/1896954,Cyprus--Kition (Extinct city),cyprus kition extinct city
2,http://id.worldcat.org/fast/1207790,Japan--Isen-chō,japan isen ch
3,http://id.worldcat.org/fast/1866544,Arizona--South Mountain Freeway,arizona south mountain freeway
4,http://id.worldcat.org/fast/1868488,England--Jump,england jump


In [92]:
geo_df.to_json("/Users/jpnelson/2020/sul-dlss/etd-abstract-similarity/data/fast-geo.json")

In [94]:
topics_df.to_json("/Users/jpnelson/2020/sul-dlss/etd-abstract-similarity/data/fast-topics.json")

In [95]:
abstracts = pd.read_pickle("/Users/jpnelson/2020/sul-dlss/etd-abstract-similarity/data/abstracts.pkl")

In [96]:
abstracts.head()

,druids,abstracts,abstracts_cleaned,departments,title
0,yq942qc7340,Metaverses are virtual worlds where users crea...,metaverses virtual worlds users create entire ...,Department of Computer Science.,Simplification algorithms for large virtual wo...
1,yx753dx0216,"In this dissertation, I use two novel test sco...",dissertation use two novel test score data se...,Graduate School of Education.,Gender disparities in U.S. educational achieve...
2,yd874rr2274,A 20-minute documentary film was created to ac...,minute documentary film created accelerate ...,Department of Geological and Environmental Sci...,"matched pair, cluster randomized, controlled t..."
3,sh260yn9550,The usage of hydrogen as an alternative energy...,usage hydrogen alternative energy carrier beco...,Department of Bioengineering.,Improving functions of redox proteins for hydr...
4,nk877ng0918,High-order methods in Computational Fluid Dyna...,high order methods computational fluid dynamic...,Department of Aeronautics and Astronautics.,analysis of stability of the flux reconstructi...


In [97]:
all_fast_df = geo_df + topics_df 

In [100]:
all_fast_df = pd.concat([geo_df, topics_df])

In [101]:
all_fast_df.head()

,URI,Label,cleaned
0,http://id.worldcat.org/fast/1891285,North America--Dalton Trail,north america dalton trail
1,http://id.worldcat.org/fast/1896954,Cyprus--Kition (Extinct city),cyprus kition extinct city
2,http://id.worldcat.org/fast/1207790,Japan--Isen-chō,japan isen ch
3,http://id.worldcat.org/fast/1866544,Arizona--South Mountain Freeway,arizona south mountain freeway
4,http://id.worldcat.org/fast/1868488,England--Jump,england jump


In [102]:
len(all_fast_df)

648999

In [105]:
all_fast_df.loc[all_fast_df['URI'] == 'http://id.worldcat.org/fast/1868488']['Label'].item()

'England--Jump'

In [106]:
from sentence_transformers import SentenceTransformer

In [107]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [108]:
abstract_embeddings = sbert_model.encode(abstracts['abstracts_cleaned'])

In [111]:
len(abstract_embeddings)

6284

In [112]:
len(abstracts['abstracts_cleaned'])

6284

In [114]:
from sklearn.cluster import KMeans

In [123]:
cluster_model = KMeans(n_clusters=30)

In [124]:
predicted_cluster = cluster_model.fit_predict(abstract_embeddings)

In [131]:
predicted_cluster[0]

3

In [126]:
len(cluster_model.labels_)

6284

In [127]:
clustered_abstracts = [[] for i in range(30)]

In [134]:
cluster_model.labels_ == predicted_cluster


array([ True,  True,  True, ...,  True,  True,  True])

In [148]:
clusters = abstracts.copy()

In [164]:
for row in clusters.iterrows():
    row[1]['cluster'] = predicted_cluster[row[0]]
    
    

In [165]:
clusters.head()

,druids,abstracts,abstracts_cleaned,departments,title
0,yq942qc7340,Metaverses are virtual worlds where users crea...,metaverses virtual worlds users create entire ...,Department of Computer Science.,Simplification algorithms for large virtual wo...
1,yx753dx0216,"In this dissertation, I use two novel test sco...",dissertation use two novel test score data se...,Graduate School of Education.,Gender disparities in U.S. educational achieve...
2,yd874rr2274,A 20-minute documentary film was created to ac...,minute documentary film created accelerate ...,Department of Geological and Environmental Sci...,"matched pair, cluster randomized, controlled t..."
3,sh260yn9550,The usage of hydrogen as an alternative energy...,usage hydrogen alternative energy carrier beco...,Department of Bioengineering.,Improving functions of redox proteins for hydr...
4,nk877ng0918,High-order methods in Computational Fluid Dyna...,high order methods computational fluid dynamic...,Department of Aeronautics and Astronautics.,analysis of stability of the flux reconstructi...


In [167]:
clusters['cluster'] = clusters.index.map(lambda x: predicted_cluster[x])

In [168]:
clusters.head()

,druids,abstracts,abstracts_cleaned,departments,title,cluster
0,yq942qc7340,Metaverses are virtual worlds where users crea...,metaverses virtual worlds users create entire ...,Department of Computer Science.,Simplification algorithms for large virtual wo...,3
1,yx753dx0216,"In this dissertation, I use two novel test sco...",dissertation use two novel test score data se...,Graduate School of Education.,Gender disparities in U.S. educational achieve...,8
2,yd874rr2274,A 20-minute documentary film was created to ac...,minute documentary film created accelerate ...,Department of Geological and Environmental Sci...,"matched pair, cluster randomized, controlled t...",8
3,sh260yn9550,The usage of hydrogen as an alternative energy...,usage hydrogen alternative energy carrier beco...,Department of Bioengineering.,Improving functions of redox proteins for hydr...,0
4,nk877ng0918,High-order methods in Computational Fluid Dyna...,high order methods computational fluid dynamic...,Department of Aeronautics and Astronautics.,analysis of stability of the flux reconstructi...,9


In [170]:
all_clusters = []
for row in clusters.groupby(by="cluster"):
    all_clusters.append(row)

In [171]:
len(all_clusters)

30

In [173]:
abstracts.head()

,druids,abstracts,abstracts_cleaned,departments,title
0,yq942qc7340,Metaverses are virtual worlds where users crea...,metaverses virtual worlds users create entire ...,Department of Computer Science.,Simplification algorithms for large virtual wo...
1,yx753dx0216,"In this dissertation, I use two novel test sco...",dissertation use two novel test score data se...,Graduate School of Education.,Gender disparities in U.S. educational achieve...
2,yd874rr2274,A 20-minute documentary film was created to ac...,minute documentary film created accelerate ...,Department of Geological and Environmental Sci...,"matched pair, cluster randomized, controlled t..."
3,sh260yn9550,The usage of hydrogen as an alternative energy...,usage hydrogen alternative energy carrier beco...,Department of Bioengineering.,Improving functions of redox proteins for hydr...
4,nk877ng0918,High-order methods in Computational Fluid Dyna...,high order methods computational fluid dynamic...,Department of Aeronautics and Astronautics.,analysis of stability of the flux reconstructi...


In [174]:
cleaned_areas = pd.read_csv("/Users/jpnelson/Downloads/Department_to_Area.csv")

In [175]:
cleaned_areas.head()

,Area,Department,Number
0,Aeronautics & Astronautics,Department of Aeronautics and Astronautics,173
1,Aeronautics & Astronautics,Department of Aeronautics & Astronautics,2
2,Anthropology,Department of Anthropology,47
3,Anthropology,Department of Anthropological Sciences,4
4,Applied Physics,Department of Applied Physics,188


In [177]:
abstracts['area'] = abstracts['departments'].apply(lambda x: cleaned_areas.loc[cleaned_areas['Department'] == x]['Area'])

ValueError: Wrong number of items passed 0, placement implies 1

In [178]:
def normalize_area(department):
    area = cleaned_areas.loc[cleaned_areas['Department'] == department]
    print(area)

In [185]:
normalize_area('Department of Aeronautics and Astronautics')

Empty DataFrame
Columns: [Area, Department, Number]
Index: []


In [188]:
cleaned_areas[cleaned_areas['Department'] == 'Department of Aeronautics and Astronautics']

,Area,Department,Number


In [190]:
cleaned_areas.tail()

,Area,Department,Number
112,Stem Cell Biology and Regenerative Medicine,Department of Stem Cell Biology and Regenerati...,1
113,Stem Cell Biology and Regenerative Medicine,Institute for Stem Cell Biology and Regenerati...,1
114,Stem Cell Biology and Regenerative Medicine,Department of Stem Cell Biology and Regenerati...,1
115,Structural Biology,Department of Structural Biology,9
116,Theater and Performance Studies,Department of Theater and Performance Studies,15


In [208]:
result = cleaned_areas.loc[cleaned_areas['Department'].str.startswith('Department of Structural Biological systesm')]

In [214]:
def test_area(department):
    department = department.replace(".", "")
    result = cleaned_areas.loc[cleaned_areas['Department'].str.startswith(department)]
    if len(result) > 0:
        return result['Area']
    else:
        print(f"Could not find Area for department {department}")
        return

In [240]:
dept_to_areas = { '': 'Unknown'}
for row in cleaned_areas.iterrows():
    dept = row[1]['Department']
    area = row[1]['Area']
    dept_to_areas[dept.strip()] = area.strip()

In [241]:
abstracts['area'] = abstracts['departments'].apply(lambda x: dept_to_areas[x.replace(".","")])

In [232]:
for key in dept_to_areas.keys():
    if 'Computer' in key:
        print(key)

Center for Computer Research in Music and Acoustics 
Computer Science Department 
Department of Computer Science 


In [236]:
dept_to_areas['Center for Computer Research in Music and Acoustics ']

'Center for Computer Research in Music and Acoustics'

In [242]:
abstracts.head()

,druids,abstracts,abstracts_cleaned,departments,title,area
0,yq942qc7340,Metaverses are virtual worlds where users crea...,metaverses virtual worlds users create entire ...,Department of Computer Science.,Simplification algorithms for large virtual wo...,Computer Science
1,yx753dx0216,"In this dissertation, I use two novel test sco...",dissertation use two novel test score data se...,Graduate School of Education.,Gender disparities in U.S. educational achieve...,Graduate School of Business
2,yd874rr2274,A 20-minute documentary film was created to ac...,minute documentary film created accelerate ...,Department of Geological and Environmental Sci...,"matched pair, cluster randomized, controlled t...","Earth, Energy and Environmental Sciences"
3,sh260yn9550,The usage of hydrogen as an alternative energy...,usage hydrogen alternative energy carrier beco...,Department of Bioengineering.,Improving functions of redox proteins for hydr...,Bioengineering
4,nk877ng0918,High-order methods in Computational Fluid Dyna...,high order methods computational fluid dynamic...,Department of Aeronautics and Astronautics.,analysis of stability of the flux reconstructi...,Aeronautics & Astronautics


In [248]:
regenerated_etds = pathlib.Path("/Users/jpnelson/2020/sul-dlss/tmp/etd-10-27/results")

In [253]:
missing_druids = []
for row in regenerated_etds.iterdir():
    druid = row.name[:-4]
    result = abstracts.loc[abstracts['druids'] == druid]
    if len(result)  < 1:
        missing_druids.append(druid)

In [254]:
len(missing_druids)

1005

In [255]:
len(missing_druids), len(abstracts)

(1005, 6284)

In [256]:
missing_druids[0]

'pt382fg9784'

In [260]:
mods = get_mods(missing_druids[0])

In [263]:
abstracts.head()

,druids,abstracts,abstracts_cleaned,departments,title,area
0,yq942qc7340,Metaverses are virtual worlds where users crea...,metaverses virtual worlds users create entire ...,Department of Computer Science.,Simplification algorithms for large virtual wo...,Computer Science
1,yx753dx0216,"In this dissertation, I use two novel test sco...",dissertation use two novel test score data se...,Graduate School of Education.,Gender disparities in U.S. educational achieve...,Graduate School of Business
2,yd874rr2274,A 20-minute documentary film was created to ac...,minute documentary film created accelerate ...,Department of Geological and Environmental Sci...,"matched pair, cluster randomized, controlled t...","Earth, Energy and Environmental Sciences"
3,sh260yn9550,The usage of hydrogen as an alternative energy...,usage hydrogen alternative energy carrier beco...,Department of Bioengineering.,Improving functions of redox proteins for hydr...,Bioengineering
4,nk877ng0918,High-order methods in Computational Fluid Dyna...,high order methods computational fluid dynamic...,Department of Aeronautics and Astronautics.,analysis of stability of the flux reconstructi...,Aeronautics & Astronautics


In [267]:
mods.find("mods:titleInfo/mods:title", namespaces=ns).text

'Inverse problems in the Pacific'

In [270]:
import sys
sys.path.append("/Users/jpnelson/2020/sul-dlss/etd-abstract-similarity/src/")
import prep

In [286]:
new_abstracts = []
start = datetime.datetime.utcnow()
print(f"Started at {start}")
errors = []
for i,druid in enumerate(missing_druids):
    record = { "druids": druid }
    mods_xml = prep.get_mods(druid)
    abstract = prep.get_abstract(mods_xml)
    if abstract is None:
        record['mods'] = mods_xml
        errors.append(record)
        continue
    record['abstracts'] = abstract
    cleaned_abstract = prep.cleanup(abstract)
    record['abstracts_cleaned'] = cleaned_abstract
    department = prep.get_department(mods_xml)
    record['departments'] = department
    try:
        area = dept_to_areas[department.replace(".","")]
    except KeyError:
        print(f"e{i}", end="")
        errors.append(record)
    record['area'] = area
    title = prep.get_title(mods_xml)
    record['title'] = title
    new_abstracts.append(record)
    if not i%10:
        print(".", end="")
    if i == 0 or not i%25:
        print(f"{i}", end="")
end = datetime.datetime.utcnow()
print(f"Finished at {end}, total time {(end-start).seconds / 60.}")

Started at 2020-10-30 19:44:10.140675
.0..25...50..75...100..125...150..175...200..225..e246.250..275...300.e317.325...350.e364.375.e387..400e402..425e429...450..475...500e501.e516.525...550..575...600..625..e643.650e659..e671675...700..725...750..775...800e807..825...850.e869.875...900..925...950e957.e965.975...1000Finished at 2020-10-30 19:50:53.321656, total time 6.716666666666667


In [279]:
dept_to_areas['Department of Medicine'] = 'Medicine'

In [278]:
dept_to_areas.pop('Medicine')

'Department of Medicine'

In [283]:
dept_to_areas['Program in Chemical Engineering'] = 'Chemical Engineering'

In [287]:
abstracts.loc[abstracts['druids']=='.DS_S']

,druids,abstracts,abstracts_cleaned,departments,title,area
895,.DS_S,,,,None,Unknown


In [288]:
len(errors)

17

In [291]:
for i,row in enumerate(errors):
    print(row.get('departments', f"Not found {i}"))

Department of Geology.
Department of Civil & Environmental Engineering Department
Not found 2
Graduate Program of Immunology.
Department of Epidemiology
Department of Computer Science Department.
Program in Chemical and Systems Biology.
Not found 7
Interdisciplinary Stem Cell Biology and Regenerative Medicine.
Earth Systems Science.
Department of Epidemiology.
Department of Civil & Environmental Engineering.
Emmet Interdisciplinary Program in Environment and Resources (Stanford University)
Department of Neurology.
Department of Civil & Environmental Engineering
Department of Cultural and Social Anthropology
Department of Health Policy.


In [292]:
len(new_abstracts)

1003

In [293]:
new_abstracts_df = pd.DataFrame(new_abstracts)

In [294]:
new_abstracts_df.head()

,druids,abstracts,abstracts_cleaned,departments,area,title
0,pt382fg9784,The settlement of Polynesia represents a uniqu...,settlement polynesia represents uniquely struc...,Institute for Computational and Mathematical E...,Computational and Mathematical Engineering,Inverse problems in the Pacific
1,hh687db3446,Catalysts are a critical component of the chem...,catalysts critical component chemical industry...,Department of Chemical Engineering,Chemical Engineering,Design and understanding of selective catalyst...
2,bg169vz8806,This dissertation examines how local governmen...,dissertation examines local government institu...,Department of Political Science,Political Science,art of not governing
3,ky788fx0492,This dissertation assesses the style of the He...,dissertation assesses style hesiodic shield he...,Department of Classics.,Classics,poetics of style in the shield of Heracles
4,kc412qs1135,Carbon mitigation policies should be designed ...,carbon mitigation policies designed balance co...,Department of Management Science and Engineering.,Management Science and Engineering,Modeling uncertain climate impacts and adaptat...


In [296]:
all_abstracts = pd.concat([abstracts, new_abstracts_df])

In [297]:
all_abstracts.head()

,druids,abstracts,abstracts_cleaned,departments,title,area
0,yq942qc7340,Metaverses are virtual worlds where users crea...,metaverses virtual worlds users create entire ...,Department of Computer Science.,Simplification algorithms for large virtual wo...,Computer Science
1,yx753dx0216,"In this dissertation, I use two novel test sco...",dissertation use two novel test score data se...,Graduate School of Education.,Gender disparities in U.S. educational achieve...,Graduate School of Business
2,yd874rr2274,A 20-minute documentary film was created to ac...,minute documentary film created accelerate ...,Department of Geological and Environmental Sci...,"matched pair, cluster randomized, controlled t...","Earth, Energy and Environmental Sciences"
3,sh260yn9550,The usage of hydrogen as an alternative energy...,usage hydrogen alternative energy carrier beco...,Department of Bioengineering.,Improving functions of redox proteins for hydr...,Bioengineering
4,nk877ng0918,High-order methods in Computational Fluid Dyna...,high order methods computational fluid dynamic...,Department of Aeronautics and Astronautics.,analysis of stability of the flux reconstructi...,Aeronautics & Astronautics


In [298]:
all_abstracts.tail()

,druids,abstracts,abstracts_cleaned,departments,title,area
998,db465vb0804,"In today's digital age, online services, such ...",today s digital age online services search e...,Computer Science Department,Protecting privacy when mining and sharing use...,Computer Science
999,sj995pm3375,This dissertation consists of three chapters. ...,dissertation consists three chapters first tw...,Graduate School of Business,Essays in microeconomics,Graduate School of Business
1000,qn448rp1399,Magnetic resonance imaging (MRI) is an importa...,magnetic resonance imaging mri important med...,Department of Electrical Engineering.,Acceleration of 3D magnetic resonance angiography,Electrical Engineering
1001,sj756gx1416,Five-year survival rates for adult B-cell acut...,five year survival rates adult b cell acute ly...,Cancer Biology Program.,Myeloid reprogramming B cell acute lymphoblast...,Cancer Biology
1002,sz929xd7180,A defining step in germ cell development is th...,defining step germ cell development exit mitot...,Cancer Biology Program.,Conserved regulation of the switch from prolif...,Cancer Biology


In [341]:
all_abstracts.to_pickle("/Users/jpnelson/2020/sul-dlss/etd-abstract-similarity/data/abstracts.pkl")
print(len(all_abstracts))

7283


In [342]:
start_emb = datetime.datetime.utcnow()
print(f"Start {start_emb}")
abstract_embeddings = sbert_model.encode(all_abstracts['abstracts_cleaned'])
end_emb = datetime.datetime.utcnow()
print(f"Finished {end_emb} total time {(end_emb-start_emb).seconds / 60.} minutes")

Start 2020-10-30 22:22:22.874855
Finished 2020-10-30 22:42:06.641384 total time 19.716666666666665 minutes


In [304]:
pickle.dump?

Signature:
pickle.dump(
    obj,
    file,
    protocol=None,
    *,
    fix_imports=True,
    buffer_callback=None,
)
Docstring:
Write a pickled representation of obj to the open file object file.

This is equivalent to ``Pickler(file, protocol).dump(obj)``, but may
be more efficient.

The optional *protocol* argument tells the pickler to use the given
protocol; supported protocols are 0, 1, 2, 3 and 4.  The default
protocol is 4. It was introduced in Python 3.4, it is incompatible
with previous versions.

Specifying a negative protocol version selects the highest protocol
version supported.  The higher the protocol used, the more recent the
version of Python needed to read the pickle produced.

The *file* argument must have a write() method that accepts a single
bytes argument.  It can thus be a file object opened for binary
writing, an io.BytesIO instance, or any other custom object that meets
this interface.

If *fix_imports* is True and protocol is less than 3, pickle will try
to 

In [344]:
with open("/Users/jpnelson/2020/sul-dlss/etd-abstract-similarity/data/abstracts-bert-embeddings.pkl", "wb+") as fo:
    pickle.dump(abstract_embeddings, fo)

,druids,abstracts,abstracts_cleaned,departments,title,area
0,yq942qc7340,Metaverses are virtual worlds where users crea...,metaverses virtual worlds users create entire ...,Department of Computer Science.,Simplification algorithms for large virtual wo...,Computer Science
1,yx753dx0216,"In this dissertation, I use two novel test sco...",dissertation use two novel test score data se...,Graduate School of Education.,Gender disparities in U.S. educational achieve...,Graduate School of Business
2,yd874rr2274,A 20-minute documentary film was created to ac...,minute documentary film created accelerate ...,Department of Geological and Environmental Sci...,"matched pair, cluster randomized, controlled t...","Earth, Energy and Environmental Sciences"
3,sh260yn9550,The usage of hydrogen as an alternative energy...,usage hydrogen alternative energy carrier beco...,Department of Bioengineering.,Improving functions of redox proteins for hydr...,Bioengineering
4,nk877ng0918,High-order methods in Computational Fluid Dyna...,high order methods computational fluid dynamic...,Department of Aeronautics and Astronautics.,analysis of stability of the flux reconstructi...,Aeronautics & Astronautics
...,...,...,...,...,...,...
6279,kc119hd5549,Small RNAs that are 19-30 nucleotides in lengt...,small rnas nucleotides length use geneti...,Department of Microbiology and Immunology,Molecular signatures of virome-host interactions,Microbiology and Immunology
6280,mz572jk7876,Computer architecture is at a critical junctur...,computer architecture critical juncture singl...,Department of Electrical Engineering,Hardware and software techniques for scalable ...,Electrical Engineering
6281,wb228xv1479,This dissertation comprises three chapters. Th...,dissertation comprises three chapters first t...,Department of Economics.,Essay in empirical industrial organization and...,Economics
6282,gv302td4683,Morphometric studies reveal that the volumes o...,morphometric studies reveal volumes many brain...,Department of Psychology,Neuroanatomical correlates of emotion reactivi...,Psychology


In [315]:
all_abstracts.drop(all_abstracts.index[[895,]])

,druids,abstracts,abstracts_cleaned,departments,title,area
0,yq942qc7340,Metaverses are virtual worlds where users crea...,metaverses virtual worlds users create entire ...,Department of Computer Science.,Simplification algorithms for large virtual wo...,Computer Science
1,yx753dx0216,"In this dissertation, I use two novel test sco...",dissertation use two novel test score data se...,Graduate School of Education.,Gender disparities in U.S. educational achieve...,Graduate School of Business
2,yd874rr2274,A 20-minute documentary film was created to ac...,minute documentary film created accelerate ...,Department of Geological and Environmental Sci...,"matched pair, cluster randomized, controlled t...","Earth, Energy and Environmental Sciences"
3,sh260yn9550,The usage of hydrogen as an alternative energy...,usage hydrogen alternative energy carrier beco...,Department of Bioengineering.,Improving functions of redox proteins for hydr...,Bioengineering
4,nk877ng0918,High-order methods in Computational Fluid Dyna...,high order methods computational fluid dynamic...,Department of Aeronautics and Astronautics.,analysis of stability of the flux reconstructi...,Aeronautics & Astronautics
...,...,...,...,...,...,...
998,db465vb0804,"In today's digital age, online services, such ...",today s digital age online services search e...,Computer Science Department,Protecting privacy when mining and sharing use...,Computer Science
999,sj995pm3375,This dissertation consists of three chapters. ...,dissertation consists three chapters first tw...,Graduate School of Business,Essays in microeconomics,Graduate School of Business
1000,qn448rp1399,Magnetic resonance imaging (MRI) is an importa...,magnetic resonance imaging mri important med...,Department of Electrical Engineering.,Acceleration of 3D magnetic resonance angiography,Electrical Engineering
1001,sj756gx1416,Five-year survival rates for adult B-cell acut...,five year survival rates adult b cell acute ly...,Cancer Biology Program.,Myeloid reprogramming B cell acute lymphoblast...,Cancer Biology


In [316]:
all_abstracts.iloc[890:896]

,druids,abstracts,abstracts_cleaned,departments,title,area
890,jv267rz6687,Cell signaling networks are complex dynamical ...,cell signaling networks complex dynamical syst...,Department of Chemical and Systems Biology.,Parameter extraction from single cell dynamics...,Chemical and Systems Biology
891,qb242rk0531,Knowledge-based approaches to planning and con...,knowledge based approaches planning control of...,Department of Electrical Engineering,Analytical goal-driven learning of procedural ...,Electrical Engineering
892,qz280bc5733,Reduced deformable bodies in the interactive a...,reduced deformable bodies interactive real tim...,Computer Science Department.,Two-way coupling of fluids to reduced deformab...,Computer Science
893,rq735dh6510,"In this thesis, we develop and use an ab initi...",thesis develop use ab initio molecular dynami...,Department of Chemistry,First principles molecular dynamics and contro...,Chemistry
894,gz312sp0723,"During normal development, tissue regeneration...",normal development tissue regeneration cancer...,Cancer Biology Program.,Dissecting the functions of cadherins in cell-...,Cancer Biology
895,.DS_S,,,,None,Unknown


In [317]:
all_abstracts.to_csv("data/all-abstracts.csv")

In [318]:
len(all_abstracts)

7287

In [330]:
all_abstracts = pd.read_csv("data/all-abstracts.csv")

In [331]:
len(all_abstracts)

7283

In [332]:
all_abstracts.head()

,Unnamed: 0,druids,abstracts,abstracts_cleaned,departments,title,area
0,0,yq942qc7340,Metaverses are virtual worlds where users crea...,metaverses virtual worlds users create entire ...,Department of Computer Science.,Simplification algorithms for large virtual wo...,Computer Science
1,1,yx753dx0216,"In this dissertation, I use two novel test sco...",dissertation use two novel test score data se...,Graduate School of Education.,Gender disparities in U.S. educational achieve...,Graduate School of Business
2,2,yd874rr2274,A 20-minute documentary film was created to ac...,minute documentary film created accelerate ...,Department of Geological and Environmental Sci...,"matched pair, cluster randomized, controlled t...","Earth, Energy and Environmental Sciences"
3,3,sh260yn9550,The usage of hydrogen as an alternative energy...,usage hydrogen alternative energy carrier beco...,Department of Bioengineering.,Improving functions of redox proteins for hydr...,Bioengineering
4,4,nk877ng0918,High-order methods in Computational Fluid Dyna...,high order methods computational fluid dynamic...,Department of Aeronautics and Astronautics.,analysis of stability of the flux reconstructi...,Aeronautics & Astronautics


In [323]:
all_abstracts.columns

Index(['Unnamed: 0', 'druids', 'abstracts', 'abstracts_cleaned', 'departments',
       'title', 'area'],
      dtype='object')

In [326]:
all_abstracts1 = all_abstracts.drop(all_abstracts['Unnamed: 0'])

KeyError: '[2177 2978 3509 ... 1000 1001 1002] not found in axis'

In [333]:
len(all_abstracts)

7283

In [335]:
all_abstracts.shape

(7283, 7)

In [336]:
del all_abstracts['Unnamed: 0']

In [339]:
len(all_abstracts)

7283

In [ ]:
import csv